<a href="https://colab.research.google.com/github/Roopana/Twitter-Ad-Analysis/blob/master/SentBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ref : https://www.sbert.net/docs/training/overview.html#creating-networks-from-scratch


In [1]:
pip install -U sentence-transformers

     |████████████████████████████████| 81kB 3.6MB/s 
     |████████████████████████████████| 2.1MB 7.8MB/s 
     |████████████████████████████████| 1.2MB 23.7MB/s 
     |████████████████████████████████| 3.3MB 38.5MB/s 
     |████████████████████████████████| 901kB 43.8MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.1.0-cp37-none-any.whl size=119615 sha256=fc28a5e8ee5031ee13326c0b13b4c9d00e6dad92b86685181671081fdf16d853
  Stored in directory: /root/.cache/pip/wheels/84/cb/21/1066bff3027215c760ca14a198f698bca8fccb92e33e2327eb
Successfully built sentence-transformers


Training Data

In [2]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

model = SentenceTransformer('distilbert-base-nli-mean-tokens')

In [3]:
import pandas as pd

df_2 = pd.read_csv("./2_man_ann_sb.csv",  index_col= None)
df_2 = df_2.dropna(subset = ['tweet_text'])

df_1 = pd.read_csv("./mann_ann_sb.csv", index_col= None)

df_3 = pd.read_csv("./3_man_ann_sb_full_1.csv", index_col = None)
df_3 = df_3.dropna(subset = ['tweet_text'])

df_raw = df_1.append(df_2).append(df_3) # using batch 1 and batch 2 for training

print(df_raw.shape)

(7500, 19)


In [4]:
# Remove ads marked as below because they are not available in ad annotations file although tweets mention them
# commercials, joe biden, pizzahut, joe bieden, michael bloomberg, mike bloomberg, scientology...
ads_remove = ['commercials', 'joe biden', 'pizzahut', 'joe bieden', 'michael bloomberg', 'mike bloomberg', 
              'scientology','papa johns',  'bakari',  'secret', 'dashlane', 'bernie the peoples perfume',
              'ram trucks', 'golden gronks', "bush's best", 'ragged old flag', 'patience', 'guitar hero',
              'disney mulan']

# ads_rename = ['nfl100', 'tide']
# rename ads with spelling faults while manually adding the annotation 

df_raw['ad_manual_adjusted'] = df_raw['ad_manual_adjusted'].apply(lambda x: x.lower())
df_raw.loc[df_raw.ad_manual_adjusted == "discover card  no we don‚äôt charge annual fees", 
       "ad_manual_adjusted"] = "discover card  no we don’t charge annual fees"
df_raw.loc[df_raw.ad_manual_adjusted == "doritos the cool ranch", 
       "ad_manual_adjusted"] = "doritos  the cool ranch"
df_raw.loc[df_raw.ad_manual_adjusted == "discover card yes we're accepted", 
       "ad_manual_adjusted"] =  "discover card  yes we’re accepted"
df_raw.loc[df_raw.ad_manual_adjusted == "discover card yes we’re accepted", 
       "ad_manual_adjusted"] =  "discover card  yes we’re accepted"
df_raw.loc[df_raw.ad_manual_adjusted == "discover card  yes we're accepted", 
       "ad_manual_adjusted"] =  "discover card  yes we’re accepted"
df_raw.loc[df_raw.ad_manual_adjusted == "budweiser typical american", 
       "ad_manual_adjusted"] = "budweiser  typical american"
df_raw.loc[df_raw.ad_manual_adjusted == 'fox  halftime show  teaser_3',
            "ad_manual_adjusted"] = "fox  halftime show  teaser_1"
df_raw.loc[df_raw.ad_manual_adjusted == 'fox  halftime show  teaser_2',
            "ad_manual_adjusted"] = "fox  halftime show  teaser_1"
            
print(df_raw.shape)
df = pd.DataFrame()
removed_Data = pd.DataFrame()

for i,row  in df_raw.iterrows():
  if row['ad_manual_adjusted'] not in ads_remove:
    df = df.append(row)
  else:
    removed_Data = removed_Data.append(row)
print(df.shape)

(7500, 19)
(7394, 19)


In [5]:
classType = 'sent_exploded' # binary or multi-class or sent-exploded

In [6]:
from nltk.corpus import stopwords
import re
import nltk
nltk.download('stopwords')

stop = stopwords.words('english')
# stop.append('superbowl')
# stop.append('super') 
# stop.append('bowl')

# remove for multi class since almost all ads have these words
if classType is not 'binary':
  stop.append('commercial')
  stop.append('ad')
  stop.append('commercials')
  stop.append('ads')
print(len(stop))

def removeMentions(text):

    textBeforeMention = text.partition("@")[0]
    textAfterMention = text.partition("@")[2]
    textAfterMention =  re.sub(r':', '', textAfterMention) #cadillac join the 31k
    tHandle = textAfterMention.partition(" ")[0].lower() #cadillac    
    text = textBeforeMention+ " " + textAfterMention  
    return text

def cleanTweet(strinp):
    strinp = re.sub(r'RT', "", strinp) # Remove RT
    strinp = strinp.lower()
    
    stop_removed_list = [word for word in strinp.split() if word not in (stop)]
    stop_removed = ' '.join([str(elem) for elem in stop_removed_list])    
    text = re.sub('https?://[A-Za-z0-9./]+', ' ', stop_removed) # Remove URLs
    text = removeMentions(text)
    text = re.sub('[^\x00-\x7F]+', ' ', text) # Remove non-ASCII chars.
    
    # remove punctuations except '-'
    punctuation = ['(', ')', '[',']','?', ':', ':', ',', '.', '!', '/', '"', "'", '@', '#', '&', '-', '_']
    text = "".join((char for char in text if char not in punctuation))
    text = re.sub('[^a-zA-Z]', ' ', text) # remove all other than alphabet chars 

#     text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text) # remove all single characters     
    stop_removed_l = [word for word in text.split() if word not in (stop)]
    stop_removed = ' '.join([str(elem) for elem in stop_removed_l]) 
    return stop_removed

print(cleanTweet("RT @cadillacabc: Joinrt the 31K james_bond") )

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
183
cadillacabc joinrt k jamesbond


In [7]:
df['text_clean'] = df['tweet_text'].apply(lambda x: cleanTweet(x))
df['ad_manual_adjusted'] = df['ad_manual_adjusted'].apply(lambda x: x.lower())
df['ad_related'] = df['ad_manual_adjusted'].apply(lambda ad: 0 if ad == 'none' else 1)

comma_filter = ~df['ad_manual_adjusted'].str.contains(',')
df = df[comma_filter]

In [8]:
df_unique = df.drop_duplicates(subset = ['text_clean'])
df_with_dupes = df
df = df_unique

print(df_with_dupes.shape)
print(df.shape)

(7332, 21)
(5824, 21)


In [9]:
ad_product_df = pd.read_csv('./SB_ad_annotations_product_category_modified.csv')
ad_product_df = ad_product_df.rename(columns = {'Ad Name': 'Ad_Name'}) # rename the column to remove space
ad_product_df = ad_product_df.dropna() # because the file has trailing empty rows, remove them
# remove fox half time show teaser_2 ad because its keywords is same as fox half time show teaser_2
print(ad_product_df.shape)
ad_product_df.drop(ad_product_df[ad_product_df['Ad_Name'] == 'FOX  Halftime Show  Teaser_2'].index, inplace = True) 
print(ad_product_df.shape)
ad_product_dict = dict()

ad_product_df['Product_modified'] = ad_product_df['Product_modified'].apply(lambda x: x.lower())
for i, row in ad_product_df.iterrows():
  ad_product_dict[row['Ad_Name'].lower()] = row['Product_modified'].lower()

ad_product_dict['none'] = 'none'
print(ad_product_dict)

df['product_modified'] = df['ad_manual_adjusted'].apply(lambda ad: ad_product_dict[ad])
df['product_modified'] = df['product_modified'].apply(lambda x: x.lower())

(75, 11)
(74, 11)
{'fast & furious 9  trailer': 'movie trailer', 'quibi  bank heist': 'tech company', 'tide  when is later  masked singer': 'laundry detergent', 'fox  a run at history  daytona 500': 'sporting event', 'donald j. trump for president  criminal justice reform': 'political campaign', 'walmart  famous visitors': 'tech company', 'marvel  black widow trailer': 'movie trailer', 'rocket mortgage  home': 'money', 'porsche  the heist': 'car', 'snickers  fix the world': 'food', 'hulu  tom brady s big announcement': 'streaming service', 'fox  chosen  lego masters': 'tv show/network', 'mountain dew': 'pop/soda', 'squarespace  winona in winona': 'tech company', 'new york life  love takes action': 'money', 'fox  super monday': 'tv show/network', 'hyundai  smaht pahk': 'car', 'cheetos  can t touch this': 'food', 'olay  make space for women': 'charitable program', 'fox  halftime show  teaser_1': 'half-time show', 'michelob  6 for 6-pack': 'charitable program/ alcohol', 'avocados from mex

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [10]:
# group the keywords of ads part of a product bucket and append these to the training sentences
ad_product_keywords_dict = ad_product_df.groupby('Product_modified')['Key Terms  Round 2'].agg(lambda x : x.sum() if x.dtype=='float64' else ' '.join(x))
# clean the ad key words - not removing duplicate words here - TODO
for ad_bucket in ad_product_keywords_dict.keys():
  ad_product_keywords_dict[ad_bucket] = cleanTweet(ad_product_keywords_dict[ad_bucket])
ad_product_df['product_modified_keywords'] = ad_product_df['Product_modified'].apply(lambda x: ad_product_keywords_dict[x])
ad_product_df.head(2)

# generate adname, ad keywords dict to use in sent exploding
ad_name_keywords_dict = dict()
for i, row in ad_product_df.iterrows():
  ad_name_keywords_dict[row['Ad_Name'].lower()] = cleanTweet(row['Key Terms  Round 2'])
ad_name_keywords_dict['none'] = 'none'
print(ad_name_keywords_dict)

ad_keywords_name_dict = dict()
for ad_name in ad_name_keywords_dict:
  keywords_temp = ad_name_keywords_dict[ad_name]
  ad_keywords_name_dict[keywords_temp] = ad_name
print(ad_keywords_name_dict)

ad_product_df['ad_name_keywords'] = ad_product_df['Ad_Name'].apply(lambda x: ad_name_keywords_dict[x.lower()])

{'fast & furious 9  trailer': 'fast furious fast furious ff f fast saga vin diesel flying truck stunts michelle rodriguez fastfurious', 'quibi  bank heist': 'quibi bank heist robbery less ten minutes quick bites big stories chance rapper megan thee stallion chancetherappermeganstallion quickbites', 'tide  when is later  masked singer': 'tide laundry laundry detergent schitt emily hampshire charlie day walts', 'fox  a run at history  daytona 500': 'fox show daytona run history great american race', 'donald j. trump for president  criminal justice reform': 'donald trump trump change unemployment stronger safer prosperous trumpforpresident donaldtrump presidentrumpobama gop trumpsupporters', 'walmart  famous visitors': 'walmart pickup spaceship toy story buzz lightyear marvin martian marvin martians arrival glass cleaners aliens men black groot lego star wars r bill bill ted flash gordon', 'marvel  black widow trailer': 'marvel black widow scarlett johansson', 'rocket mortgage  home': 'qu

In [11]:
# use column name 'ad_manual_adjusted' of df to get ad_name
def get_ad_related_twts(df, removeCommas = True):
  df['ad_manual_adjusted'] = df['ad_manual_adjusted'].apply(lambda x: x.lower())
  ad_filter = df['ad_manual_adjusted']!= 'none'
  ad_related_twts = df[ad_filter]
  if removeCommas:
    ad_filter_1 = ~ad_related_twts['ad_manual_adjusted'].str.contains(',')
    ad_related_twts = ad_related_twts[ad_filter_1]
  return ad_related_twts

def getAdTweets(ad_related_twts, ad):
  return ad_related_twts[ad_related_twts.ad_manual_adjusted == ad].shape[0]

def get_ad_id_dict(ad_related_twts): 
  n_ad_related = ad_related_twts.shape[0]
  print("# ad related tweets: "+ str(n_ad_related))
  ads_annotated = ad_related_twts.ad_manual_adjusted.values
  adset = set(ads_annotated)
  print("unique ads:"+ str(len(adset)))
  ad_id_dict = {}
  i = 0
  for ad in adset : 
    if(getAdTweets(ad_related_twts, ad) >=2):
      ad_id_dict[ad] = i
      i = i+1
    else:
      print('ad with <2 samples: '+ str(ad))
  print(" No of ads with >=2 samples:"+ str(len(ad_id_dict)))
  ad_id_dict['none'] = len(ad_id_dict)
  print(ad_id_dict)
  return ad_id_dict

def convertAdNameToAdId(ad_id_dict, ad_name):
  if ad_name in ad_id_dict:
    return ad_id_dict[ad_name]
  else:
    return ad_id_dict['none']

ad_related_twts = get_ad_related_twts(df)
ad_id_dict = get_ad_id_dict(get_ad_related_twts(df))
n_unique_ads = len(ad_id_dict) # ad_id_dict has none as well, so minus 1 when using embeddings
df['ad_manual_adjusted_id'] = df['ad_manual_adjusted'].apply(lambda x: convertAdNameToAdId(ad_id_dict,x))
print(n_unique_ads)


# ad related tweets: 2597
unique ads:63
ad with <2 samples: tide  ww
ad with <2 samples: discover card  no we don’t charge annual fees
ad with <2 samples: fast & furious 9  trailer
 No of ads with >=2 samples:60
{'planters  baby funeral': 0, 'hummer  gmc  quiet revolution': 1, 'heinz ketchup  find the goodness  four at once': 2, 'reese s  rock': 3, 'pop-tarts  pop-tarts fixed the pretzel commercial': 4, 'turbotax  turbotax  all people are tax people remix': 5, 'microsoft surface  be the one': 6, 'genesis  going away party': 7, 'fox  toads  the masked singer': 8, 'weathertech  lucky dog': 9, 'michelob  6 for 6-pack': 10, 'facebook  ready to rock?': 11, 'new york life  love takes action': 12, 'nfl  inspire change  anquan boldin': 13, 'audi  let it go [t1]': 14, 'hyundai  smaht pahk': 15, 'doritos  the cool ranch': 16, 'pringles  the infinite dimensions of rick and morty': 17, 'marvel  black widow trailer': 18, 'tide  finally later': 19, 'avocados from mexico  the avocados from mexico sho

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


61


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [12]:
from sentence_transformers import util

tweets = df['text_clean'].values
ad_names = df['ad_manual_adjusted'].values
ad_keywords = list(ad_name_keywords_dict.values())

twt_embeddings = model.encode(tweets, convert_to_tensor=True)
ad_embeddings = model.encode(ad_keywords, convert_to_tensor=True)

#Compute cosine-similarities for each sentence with each other sentence
cosine_scores = util.pytorch_cos_sim(twt_embeddings, ad_embeddings)

#Find the pairs with the highest cosine similarity scores
pairs = []
for i in range(len(cosine_scores)-1):
    max_ad_keywords = ''
    max_score = 0
    ad_sentBERT = ''
    max_score_ad_id = 0
    for j in range(0, len(ad_embeddings)):
        if(cosine_scores[i][j] > max_score):
          max_score = cosine_scores[i][j]
          max_ad_keywords = ad_keywords[j]
          ad_sentBERT = ad_keywords_name_dict[max_ad_keywords]
          max_score_ad_id = j
        # pairs.append({'index': [i, j], 'score': cosine_scores[i][j],'tweet': tweets[i],  'ad_sentBERT_keywords': curr_ad_keywords,'ad_sentBERT': ad_keywords_name_dict[curr_ad_keywords] })
    pairs.append({'index': [i, max_score_ad_id], 'score': max_score,'tweet': tweets[i],  'ad_sentBERT_keywords': max_ad_keywords,'ad_sentBERT': ad_sentBERT })
#Sort scores in decreasing order
#pairs = sorted(pairs, key= lambda x: x['score'], reverse=True)

for pair in pairs[0:10]:
    i, j = pair['index']
    print("{} \t\t {} \t\t Score: {:.4f}".format(tweets[i], ad_keywords[j], pair['score']))

man wanna look brightside thought fringe playoff team year started shit 		 fox super monday office toby toby flenderson 		 Score: 0.6268
harphampeg lynnelanae wgravelee darngoood franklingraham pepsi idea feel female given 		 pop culture discover credit card mean girls ted noannualfeediscovercredit nodiscover 		 Score: 0.7167
footballpei see available silent auction tonight super bowl party please go facebook page 		 inspire change police shootings black men nfl inspirechange 		 Score: 0.6939
budweiserusa party city heat superbowlliv weekend miami giving away limitededition bud bow 		 rick morty adult swim pringles pickle rick rick morty new flavor summer rickmorty adultswim 		 Score: 0.7421
pair localjackson products getting superbowl rings callawaycharge breeland speaks bspeaks 		 pop culture annual fee discover credit card chandler friends jack black mike myers austin powersnoannualfeediscovercredit yestodiscover 		 Score: 0.7406
stansdad j lo better sing favorite song halftimeshow 

In [13]:
df_pairs = pd.DataFrame(pairs, columns=['index', 'score', 'tweet', 'ad_sentBERT_keywords', 'ad_sentBERT'])

In [14]:
df['ad_sentBERT'] = df_pairs['ad_sentBERT']
df['score'] = df_pairs['score']
df['match'] = df.apply(lambda row: 1 if row['ad_sentBERT'] == row['ad_manual_adjusted'] else 0, axis = 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [15]:
df['ad_sentBERT'].head()

0                                    fox  super monday
1                    discover card  yes we’re accepted
2                   nfl  inspire change  anquan boldin
3    pringles  the infinite dimensions of rick and ...
4        discover card  no we don’t charge annual fees
Name: ad_sentBERT, dtype: object

In [16]:
print(df.shape)
print(df['match'].sum())
print(df['match'].sum()*100/ df.shape[0])
df.to_csv('./sent_bert_results.csv')

(5824, 26)
109
1.871565934065934


**Train the model**

Loss Function - CosineSimilarityLoss

In [17]:
# train_examples = [InputExample(texts=['My first sentence', 'My second sentence'], label=0.8),
#    InputExample(texts=['Another pair', 'Unrelated sentence'], label=0.3)]

# train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
# train_loss = losses.CosineSimilarityLoss(model)

# model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100, evaluator=evaluator, evaluation_steps=500)
# #           ,output_path=model_save_path)

NameError: ignored

**Evaluate model**

In [ ]:
# from sentence_transformers import evaluation
# sentences1 = ['This list contains the first column', 'With your sentences', 'You want your model to evaluate on']
# sentences2 = ['Sentences contains the other column', 'The evaluator matches sentences1[i] with sentences2[i]', 'Compute the cosine similarity and compares it to scores[i]']
# scores = [0.3, 0.6, 0.2]

# evaluator = evaluation.EmbeddingSimilarityEvaluator(sentences1, sentences2, scores)
# #evaluator = EmbeddingSimilarityEvaluator.from_input_examples(sts_reader.get_examples('sts-dev.csv'))
